# SpikeInterface DEMO v0.101 - Edinburgh - May 2024

For this demo we will use a small file provided by Eduarda Centeno, Arthur Leblois and Aude Retailleau from IMN lab in Bordeaux.

It is a recording from a zebra finch for testing the probe ASSY-236-H5 from cambridge neurotech.

![](img/zebra_finch.jpeg)

The recording system is the openephys usb3 board.


This file is only for testing or teaching purposes.

# Table of contents

* [0. Preparation](#preparation)
* [1. Reading recording and sorting](#loading)
* [2. Preprocessing](#preprocessing)
* [3. Saving and loading SpikeInterface objects](#save-load)
* [4. Spike sorting](#spike-sorting)
* [5. Postprocessing and SortingAnalyzer](#postprocessing)
* [6. Validation and curation](#curation)
* [7. Viewers](#viewers)
* [8. Spike sorting comparison](#comparison)
* [9. Exporters](#exporters)

# 0. Preparation <a class="anchor" id="preparation"></a>

### Download the ephys data

Tthe data `cambridgeneurotech_openephys_recording.zip` can be downloaded from here:

https://drive.google.com/drive/folders/17RlgsMLheW82IMLMgmTFifVACebDZ8X5?usp=sharing



In [ ]:
# spikeinterface has manu submodules: here this is the lazy way to import all of then!
import spikeinterface.full as si

In [ ]:
print(f"SpikeInterface version: {si.__version__}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

import warnings
warnings.simplefilter("ignore")

# %matplotlib widget
%matplotlib inline

# 1. Reading recording and sorting <a class="anchor" id="loading"></a>

In [ ]:
base_folder = Path('/home/samuel/DataSpikeSorting/Edinburgh_SI_tutorials/')
oe_folder = base_folder / "openephys_recording" / "2023-08-23_15-56-05"

In [ ]:
full_raw_rec = si.read_openephys(oe_folder)

The `read_openephys()` function returns a `Recording` (or `RecordingExtractor`) object. We can print it to visualize some of its properties:

In [ ]:
print(full_raw_rec)

In [ ]:
full_raw_rec

`spikeinterface.extractors` integrate many readers:

  * `read_nwb()`
  * `read_blackrock()`
  * `read_spikeglx()`
  * ...


For a full list see the documentation [here](https://spikeinterface.readthedocs.io/en/latest/modules/extractors.html).

A `Recording` object extracts information about channel ids, channel locations (if present), the sampling frequency of the recording, and the extracellular traces (when prompted).

SpikeInterface supports multi-segment recordings. A segment is a contiguous piece of data, and sometimes recordings can be made of multiple acquisitions, for examples a baseline, a stimulation phase, and a post recording. In such cases, the recording object will be made of multiple segments and be treated as such over the pipeline.

The `get_traces()` function returns a TxN numpy array where N is the number of channel ids passed in (all channel ids are passed in by default) and T is the number of frames (determined by start_frame and end_frame).

In [ ]:
fs = full_raw_rec.get_sampling_frequency()
trace_snippet = full_raw_rec.get_traces(start_frame=int(fs*0), end_frame=int(fs*2))

In [ ]:
print('Traces shape:', trace_snippet.shape)

Before moving on with the analysis, we have to load the probe information. For this we will use the [ProbeInterface](https://probeinterface.readthedocs.io/en/main/index.html) package. 

ProbeInterface allows to easily create, manipulate, and visualize neural probes. Moreover, it comes with a wide range of IO functions to import and export existing formats. Finally, we have created a public library of commercial probes (https://github.com/SpikeInterface/probeinterface_library) that can be retrieved with a single line of code.

Let's import `probeinterface`, download the probe and plot it!

In [ ]:
import probeinterface as pi
from probeinterface.plotting import plot_probe

In [ ]:
manufacturer = 'cambridgeneurotech'
probe_name = 'ASSY-236-H5'

probe = pi.get_probe(manufacturer, probe_name)
print(probe)

In most experiments, the neural probe has a connector, that is interfaced to an headstage, which in turn connects to the acquisition system. This *pathway* usually results in a channel remapping, which means that the order of the contacts on the probe is different than the order of the recorded traces.

`probeinterface` provides a growing collection of common pathways that can be loaded directly to wire a device and apply the correct channel mapping:

In [ ]:
pi.get_available_pathways()

In [ ]:
probe.wiring_to_device('cambridgeneurotech_mini-amp-64')

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10))
w = plot_probe(probe, ax=ax, with_contact_id=True, with_device_index=True)
ax.set_xlim(-100, 100)
ax.set_ylim(-50, 300)

The probe now has contact ids `id#` and device ids `dev#`! The `device_channel_index` is the index of the trace corresponding to the contact. For example, the bottom left contact is the 62-th signal in the recording.

When loading the probe, the device indices (and all the other contact properties) are automatically sorted.
And **very importantly** the recording is reduced to 64 channels, because the other 8 channels are auxiliary channels.

In [ ]:
raw_rec = full_raw_rec.set_probe(probe)
raw_rec

We can now visualize the `channel_id` (channel name) from Open Ephys.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
w = si.plot_probe_map(raw_rec, with_channel_ids=True, with_contact_id=True, ax=ax)
ax.set_xlim(-100, 100)
ax.set_ylim(-50, 300)

The `widgets` module includes several convenient plotting functions that can be used to explore the data:

In [ ]:
%matplotlib widget
w = si.plot_traces(raw_rec, backend="ipywidgets", mode='map')

### Properties 

`Recording` objects can have *properties*. A property is a piece of information attached to a channel, e.g. group or location.

Similarly, for `Sorting` objects (that we'll cover later), anything related to a unit can be stored as a property. 

We can check which properties are in the extractor as follows:

In [ ]:
print("Properties:\n", list(raw_rec.get_property_keys()))

We can also specify a property on a subset of channels. In this case, the non-specified channels will be filled empty values:

In [ ]:
raw_rec.set_property(key='quality', values=["good"]*(raw_rec.get_num_channels() - 3),
                     ids=raw_rec.get_channel_ids()[:-3])
raw_rec

In [ ]:
raw_rec.get_property("quality")[-10:]

### Annotations

*Annotations* can be attached to any object and they can carry any information related to the recording or sorting objects.

Let's add an annotation about this tutorial:

In [ ]:
raw_rec.annotate(description="Dataset for SI tutorial")
raw_rec

In [ ]:
print(raw_rec.get_annotation_keys())

# 2. Preprocessing <a class="anchor" id="preprocessing"></a>


Now that the probe information is loaded we can do some preprocessing using `preprocessing` module.

We can filter the recordings, rereference the signals to remove noise, discard noisy channels, whiten the data, remove stimulation artifacts, etc. (more info [here](https://spiketoolkit.readthedocs.io/en/latest/preprocessing_example.html)).

For this notebook, let's filter the recordings and apply common median reference (CMR). 
**IMPORTANT:** All preprocessing functions return new `Recording` objects that apply the underlying preprocessing operation when requested. This allows users to access the preprocessed data in the same way as the raw data.

Below, we bandpass filter the recording and apply common median reference to the original recording:

In [ ]:
recording_f = si.bandpass_filter(raw_rec, freq_min=300, freq_max=9000)
print(recording_f)

Let's now apply Common Median Reference (CMR):

In [ ]:
recording_cmr = si.common_reference(recording_f, reference='global', operator='median')
print(recording_cmr)

We can plot the traces after applying CMR:

In [ ]:
recording_layers = dict(
    filt=recording_f,
    common=recording_cmr
)

w = si.plot_traces(recording_layers, mode='map', time_range=[10, 10.1], backend="ipywidgets")

In [ ]:
%gui qt
w = si.plot_traces(recording_layers, mode='line', time_range=[10, 10.1], backend="ephyviewer")

The previous plot clearly shows some strange channels on the border.

We can detect then and remove then from the recording using the `detect_bad_channels` function.

In [ ]:
bad_channel_ids, bad_channel_labels = si.detect_bad_channels(recording_f, method='coherence+psd')
print(bad_channel_ids)
print(bad_channel_labels)

In [ ]:
recording_f_good = recording_f.remove_channels(bad_channel_ids)
print(recording_f_good)

In [ ]:
recording_cmr_good = si.common_reference(recording_f_good, reference='global', operator='median')
print(recording_cmr_good)

In [ ]:
%matplotlib widget
w = si.plot_traces(recording_cmr_good, mode='map',time_range=[10, 10.1], backend="ipywidgets")

## Take only 5 min. for demo

Since we are going to spike sort the data, let's first cut out a 5-minute recording, to speed up computations.

We can easily do so with the `frame_slice()` function:

In [ ]:
fs = recording_cmr_good.get_sampling_frequency()
recording_shorten = recording_cmr_good.frame_slice(start_frame=0*fs, end_frame=300*fs)
recording_shorten

# 3. Saving and loading SpikeInterface objects <a class="anchor" id="save-load"></a>

All operations in SpikeInterface are *lazy*, meaning that they are not performed if not needed. This is why the creation of our filter recording was almost instantaneous. However, to speed up further processing, we might want to **save** it to a file and perform those operations (eg. filters, CMR, etc.) at once. 


In [ ]:
si.set_global_job_kwargs(n_jobs=-1, chunk_duration="1s", progress_bar=True)

In [ ]:
if (base_folder / "preprocessed").is_dir():
    recording_saved = si.load_extractor(base_folder / "preprocessed")
else:
    recording_saved = recording_shorten.save(folder=base_folder / "preprocessed")

After saving the SI object, we can easily load it back in a new session:

In [ ]:
recording_saved = si.load_extractor(base_folder / "preprocessed")

In [ ]:
recording_saved

**IMPORTANT**: the same saving mechanisms are available also for all SortingExtractor

# 4. Spike sorting <a class="anchor" id="spike-sorting"></a>

We can now run spike sorting on the above recording. We will use different spike sorters for this demonstration, to show how easy SpikeInterface makes it easy to interchengably run different sorters :)

Let's first check the available and installed sorters in `SpikeInterface`.
We will sort the bandpass cached filtered recording the `recording_saved` object.

In [ ]:
si.available_sorters()

In [ ]:
si.installed_sorters()

The `spikeinterface.sortingcomponents` module includes functions that can be used to create custom spike sorting pipelines built-in in `SpikeInterface`. It is still experimental and under heavy development, but there are already 3  SI-based sorters available:

* `tridesclous2` (developed by Samuel Garcia)
* `spykingcircus2` (developed by Pierre Yger)
* `simple` which is only for demo and teaching

They can be run with the same `run_sorter` function, but they don't require any additional installation!

In [ ]:
si.run_sorter?

We can retrieve the parameters associated to any sorter with the `get_default_params()` function from the `sorters` module:

In [ ]:
si.get_default_sorter_params('kilosort2_5')

To modify a parameter, we can easily pass it to the `run` function as an extra argument!

### Run sorter locally

In [ ]:
# si.Kilosort2_5Sorter.set_kilosort2_5_path('/home/samuel.garcia/Documents/SpikeInterface/code_sorters/Kilosort2.5/')
si.installed_sorters()

In [ ]:
sorter_params = {'do_correction': False}

```python
sorting_KS25 = si.run_sorter('kilosort2_5', recording_saved,
                             output_folder=base_folder / 'sorter_KS25',
                             verbose=True, **sorter_params)
```

We can check the output object:

In [ ]:
sorting_KS25 = si.read_sorter_folder(base_folder / 'sorter_KS25')
sorting_KS25

SpikeInterface ensures full provenance of the spike sorting pipeline. Upon running a spike sorter, a `spikeinterface_params.json` file is saved in the `output_folder`. This contains a `.json` version of the recording and all the input parameters. 

In [ ]:
!ls {base_folder}/sorter_KS25

In [ ]:
!cat {base_folder}/sorter_KS25/spikeinterface_params.json

The spike sorting returns a `Sorting` object. Let's see some of its functions:

In [ ]:
print(f'Spike train of a unit: {sorting_KS25.get_unit_spike_train(unit_id=1)}')
print(f'Spike train of a unit (in s): {sorting_KS25.get_unit_spike_train(unit_id=1, return_times=True)}')

We can use `spikewidgets` functions for some quick visualizations:

In [ ]:
%matplotlib inline
w_rs = si.plot_rasters(sorting_KS25, time_range=(50., 70.))

We can also save a spike sorting output for future use:

In [ ]:
sorting_saved_KS25 = sorting_KS25.save(folder=base_folder / "sorting_KS25")

### Run sorter in container

Some sorters are hard to install! To alleviate this headache, SI provides a built-in mechanism to run a spike sorting job in a docker container.

We are maintaining a set of sorter-specific docker files in the [spikeinterface-dockerfiles repo](<https://github.com/SpikeInterface/spikeinterface-dockerfiles>)
and most of the docker images are available on Docker Hub from the [SpikeInterface organization](<https://hub.docker.com/orgs/spikeinterface/repositories>).

Running spike sorting in a docker container just requires to:

1. have docker/singularity installed
2. have docker/singularity python SDK installed (`pip install docker/spython`)

When docker/singularity is installed, you can simply run the sorter in a container image:



```python
# run spike sorting on entire recording
sorting_KS2 = si.run_sorter('kilosort2', recording_saved, 
                            output_folder=base_folder / 'sorter_KS2',
                            verbose=True,
                            docker_image=True, 
                            **job_kwargs)
```

In [ ]:
sorting_KS2 = si.read_sorter_folder(base_folder / 'sorter_KS2')
sorting_KS2

```python
sorting_KS4 = si.run_sorter('kilosort4', recording_saved, 
                            output_folder=base_folder / 'sorter_KS4',
                            verbose=True,
                            # sorter params
                            do_correction=False,
                )
```


In [ ]:
sorting_KS4 = si.read_sorter_folder(base_folder / 'sorter_KS4')
sorting_KS4


# 5. Postprocessing and SortingAnalyzer <a class="anchor" id="postprocessing"></a>

The core of postprocessing spike sorting results need to paired recording-sorting objects.

In the `spikeinterface` API, `SortingAnalyzer` class in the `core` module is the base for any postprocessing.

The `SortingAnalyzer` object handles a collection of extension to compute additional data and inspect the quality of the sorting:
  * waveforms
  * templates
  * spike amplitudes
  * quality metrics
  * ...

In [ ]:
recording_saved = si.load_extractor(base_folder / "preprocessed")
sorting = sorting_KS25
print(sorting)

In [ ]:
analyzer = si.create_sorting_analyzer(sorting, recording_saved, sparse=True)
print(analyzer)

Let's start with a few extensions:

In [ ]:
# the "random_spikes" extension selects a subset of spikes to compute subsequent extensions
analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
analyzer.compute("waveforms", ms_before=1.2, ms_after=2.5)
analyzer.compute("templates", operators=["average", "std"])
analyzer.compute("noise_levels")

analyzer

Now waveforms are computed and stored in the provided `waveforms` extension. We can now retrieve waveforms and templates easily:

In [ ]:
wf_ext = analyzer.get_extension("waveforms")

In [ ]:
waveforms0 = wf_ext.get_waveforms_one_unit(unit_id=0)
print(f"Waveforms shape: {waveforms0.shape}")
waveforms1 = wf_ext.get_waveforms_one_unit(unit_id=1)
print(f"Waveforms shape: {waveforms1.shape}")


In [ ]:
template_ext = analyzer.get_extension("templates")

In [ ]:
templates_array = template_ext.get_templates(operator="average")
print(type(templates_array), templates_array.shape)




The `SortingAnalyzer` is also compatible with several `widgets` to visualize the spike sorting output:

We can also render interactive plots with the `ipywidgets` backend!

In [ ]:
%matplotlib widget
w = si.plot_unit_templates(analyzer, backend="ipywidgets")

## Sparsity

Especially when working with silicon high-density probes, or when our probe has multiple groups (e.g. multi-shank, tetrodes), we don't care about waveform/templates on *all* channels, but only on a subset of relevant channels for each unit. We refer to this subsets as **sparsity**.

By default, the `SortingAnalyzer` computes and uses a sparsity based on "distance", but we can also compute it with other options and set it manually:

In [ ]:
sparsity = si.estimate_sparsity?

In [ ]:
sparsity_small = si.estimate_sparsity(recording_saved, sorting, num_spikes_for_sparsity=200, method="radius", radius_um=100.)
sparsity_small

In [ ]:
sparsity_large = si.estimate_sparsity(recording_saved, sorting, num_spikes_for_sparsity=200, method="radius", radius_um=250.)
sparsity_large

In [ ]:
sparsity_best = si.estimate_sparsity(recording_saved, sorting, num_spikes_for_sparsity=200, method="best_channels", num_channels=8)
sparsity_best

In [ ]:
for unit_id in sparsity_best.unit_ids[::30]:
    print(unit_id, list(sparsity_best.unit_id_to_channel_ids[unit_id]))

In [ ]:
# we can use this sparsity object to create an other analyzer
analyzer_other_sparsity = si.create_sorting_analyzer(sorting, recording_saved, sparsity=sparsity_best)
analyzer_other_sparsity

In [ ]:
analyzer_other_sparsity.sparsity

Most of the plotting, computation and export functions are using this `sparsity` in the background!

## Extensions tour

In [ ]:
si.get_available_analyzer_extensions()

In [ ]:
si.get_default_analyzer_extension_params("correlograms")

### Spike amplitudes

Spike amplitudes can be computed with the `get_spike_amplitudes` function.

In [ ]:
analyzer.compute("spike_amplitudes")

In [ ]:
%matplotlib widget
w = si.plot_amplitudes(analyzer, backend="ipywidgets")

### Compute unit and spike locations

When using silicon probes, we can estimate the unit (or spike) location with triangulation. This can be done either with a simple center of mass or by assuming a monopolar model:

$$V_{ext}(\boldsymbol{r_{ext}}) = \frac{I_n}{4 \pi \sigma |\boldsymbol{r_{ext}} - \boldsymbol{r_{n}}|}$$

where $\boldsymbol{r_{n}}$ is the position of the neuron, and $\boldsymbol{r_{n}}$ of the electrode(s).

In [ ]:
analyzer.compute("unit_locations", method="monopolar_triangulation")
analyzer.compute("spike_locations", method="center_of_mass")

In [ ]:
%matplotlib widget
w = si.plot_unit_locations(analyzer, backend="ipywidgets")

In [ ]:
%matplotlib widget
w = si.plot_spike_locations(analyzer, max_spikes_per_unit=300, backend="ipywidgets")

## Template similarity

In [ ]:
analyzer.compute("template_similarity")

In [ ]:
si.plot_template_similarity(analyzer)

### Compute correlograms

In [ ]:
analyzer.compute("correlograms")

In [ ]:
si.plot_autocorrelograms(analyzer, unit_ids=sorting.unit_ids[::30])
si.plot_crosscorrelograms(analyzer, unit_ids=sorting.unit_ids[::30])

### Principal components

PCA scores can be easily computed with the `"principal_components"` extension. Similarly to the `"waveforms"`, the function compute the components only on some random spikes.

Can be done with `si.compute_principal_components(analyzer, ...)` or `analyzer.compute("principal_components", ...)`




In [ ]:
analyzer.compute("principal_components", n_components=3, mode="by_channel_global")
analyzer

### Compute template metrics

Template metrics, or extracellular features, such as peak to valley duration or full-width half maximum, are important to classify neurons into putative classes (excitatory - inhibitory). The `postprocessing` allows one to compute several of these metrics:

In [ ]:
print(si.get_template_metric_names())

In [ ]:
# analyzer.compute("template_metrics")
template_metrics = si.compute_template_metrics(analyzer)
display(template_metrics)

In [ ]:
w = si.plot_template_metrics(analyzer, include_metrics=["peak_to_valley", "half_width"], backend="ipywidgets")

For more information about these template metrics, we refer to this [documentation](https://github.com/AllenInstitute/ecephys_spike_sorting/tree/master/ecephys_spike_sorting/modules/mean_waveforms) from the Allen Institute.

# 6. Quality metrics and curation <a class="anchor" id="curation"></a>

The `qualitymetrics` module also provides several functions to compute qualitity metrics to validate the spike sorting results.

Let's see what metrics are available:

In [ ]:
print(si.get_quality_metric_list())
print(si.get_quality_pca_metric_list())

In [ ]:
metric_names = si.get_quality_metric_list()
qm = si.compute_quality_metrics(analyzer, metric_names=metric_names, verbose=True)

In [ ]:
display(qm)

For more information about these waveform features, we refer to the [SpikeInterface documentation](https://spikeinterface.readthedocs.io/en/latest/module_qualitymetrics.html) and to this excellent [documentation](https://allensdk.readthedocs.io/en/latest/_static/examples/nb/ecephys_quality_metrics.html) from the Allen Institute.

## Curation

Very often Kilosort is finding a unit severals times, which are also called "duplicate" units.

`spikeinterface` has a convenient function to find and remove these units:

In [ ]:
sorting_clean = si.remove_redundant_units(analyzer)
print(analyzer.sorting)
print(sorting_clean)

### Automatic curation based on quality metrics

A viable option to curate (or at least pre-curate) a spike sorting output is to filter units based on quality metrics. As we have already computed quality metrics a few lines above, we can simply filter the `qm` dataframe based on some thresholds.

Here, we'll only keep units with an ISI violation threshold < 0.5 and amplitude cutoff < 0.1:

In [ ]:
isi_viol_thresh = 0.5
amp_cutoff_thresh = 0.1

A straightforward way to filter a pandas dataframe is via the `query`.
We first define our query (make sure the names match the column names of the dataframe):

In [ ]:
our_query = f"amplitude_cutoff < {amp_cutoff_thresh} & isi_violations_ratio < {isi_viol_thresh}"
print(our_query)

and then we can use the query to select units:

In [ ]:
keep_units = qm.query(our_query)
keep_unit_ids = keep_units.index.values
keep_unit_ids

In [ ]:
sorting_auto_KS25 = sorting.select_units(keep_unit_ids)
print(f"Number of units before curation: {len(sorting.get_unit_ids())}")
print(f"Number of units after curation: {len(sorting_auto_KS25.get_unit_ids())}")

We can also save all the waveforms and post-processed data for curated units in a separate folder:

In [ ]:
analyzer_curated = analyzer.select_units(keep_unit_ids)
analyzer_curated

## Save the SortingAnalyzer

There are two possible formats one can save the `SortingAnalyzer` to: `binary_folder` and `zarr`.

Note that the `save_as()` function returns a new `SortingAnalyzer` object, which is now bound to a location on disk.
This means that additional extensions can be directly saved to disk.

In [ ]:
analyzer_saved = analyzer.save_as(folder=base_folder / "analyzer_KS25", format="binary_folder")

analyzer_curated_saved = analyzer_curated.save_as(folder=base_folder / "analyzer_KS25_curated.zarr", format="zarr")

print(analyzer_saved)
print(analyzer_curated_saved)

We can easily reload a `SortingAnalyzer` later on as follows:

In [ ]:
analyzer_curated = si.load_sorting_analyzer(base_folder / "analyzer_KS25_curated.zarr")
analyzer_curated

# 7. Viewers <a class="anchor" id="viewers"></a>


### SpikeInterface GUI

A QT-based GUI built on top of SpikeInterface objects.

Developed by Samuel Garcia, CRNL, Lyon.

This can be run directly in a termnal with

```bash
sigui /path/to/analyzer
```

or from python

In [ ]:
%gui qt
si.plot_sorting_summary(analyzer_curated, backend="spikeinterface_gui")

### Sorting Summary - SortingView

The `sortingview` backend requires an additional step to configure the transfer of the data to be plotted to the cloud. 

See documentation [here](https://spikeinterface.readthedocs.io/en/latest/module_widgets.html).

Developed by Jeremy Magland and Jeff Soules, Flatiron Institute, NYC

In [ ]:
w = si.plot_sorting_summary(analyzer_curated, backend="sortingview", curation=True)

# 9. Spike sorting comparison <a class="anchor" id="comparison"></a>

Do the outputs of different sorters agree with each other?

To answer this question we can use the `comparison` module.

### Compare two sorters

In [ ]:
comp_KS2_KS25 = si.compare_two_sorters(sorting_KS2, sorting_KS25, 'KS2', 'KS25')

In [ ]:
w = si.plot_agreement_matrix(comp_KS2_KS25)

### Compare multiple sorters

In [ ]:
si.compare_multiple_sorters?

In [ ]:
multi_comp = si.compare_multiple_sorters(
    sorting_list=[sorting_KS2, sorting_KS25, sorting_KS4],
    name_list=['KS2', 'KS25', 'KS4'],
    spiketrain_mode='union',
    verbose=True
)

In [ ]:
w = si.plot_multicomparison_agreement(multi_comp)
w = si.plot_multicomparison_agreement_by_sorter(multi_comp)

# 10. Exporters <a class="anchor" id="exporters"></a>

## Export to Phy for manual curation

To perform manual curation we can also export the data to [Phy](https://github.com/cortex-lab/phy):

```python
si.export_to_phy(analyzer, output_folder=base_folder / 'phy_KS25', 
                   compute_amplitudes=False, compute_pc_features=False, copy_binary=False,
                   )
```

After curating the results we can reload it using the `read_phy` and exclude the units that we labeled as `noise`:

```python
sorting_phy_curated = si.read_phy(base_folder / 'phy_KS25/', exclude_cluster_groups=['noise'])
```

## Export a "spike sorting" report to a folder

This export function creates a bunch figures figures that summarize the sorting results:

In [ ]:
si.export_report(analyzer_curated, output_folder=base_folder / 'report_KS25_curated')

### Et voilà!